# F2Heal - PAC analysis



In [ ]:
# SPDX-License-Identifier: AGPL-3.0-or-later

Install required python packages, missing from std Colab install.

In [ ]:
%pip install mne mne_bids pactools openneuro-py autoreject

Start of the python script. Import required libraries.

In [ ]:
import mne
import mne_bids
import openneuro

import autoreject

import numpy as np
import pactools
import matplotlib.pyplot as plt

Configurable parameter. Change to `False` for less output.

In [ ]:
verbose = True

A class with configuration data describing the OpenNeuro dataset that will be used from here on.

In [ ]:
class ONDataset:
    """
    Open EEG dataset from openneuro
    Parkinson's EEG dataset <https://openneuro.org/datasets/ds002778
    """
    dataset = 'ds002778'
    bids_root = "/tmp/mne"
    datatype = 'eeg'
    task = 'rest'
    suffix = 'eeg'


A class with configuration parameters relevant for the PAC generation

In [ ]:
class PACSettings:
    low_fq_range = np.linspace(13,50,150)
    low_fq_width = 2
    high_fq_range= np.linspace(50,200,150)
    high_fq_width= 4
    method= 'duprelatour'

    def create_Comodulogram(fs):
        return pactools.Comodulogram (fs=fs,
                                      low_fq_range=PACSettings.low_fq_range,
                                      low_fq_width=PACSettings.low_fq_width,
                                      high_fq_range=PACSettings.high_fq_range,
                                      high_fq_width=PACSettings.high_fq_width,
                                      method=PACSettings.method,
                                      progress_bar=verbose,
                                      n_jobs=16)

Pick a valid subject from list: 3, 5, 6, 9, 11, 12, 13, 14, 16, 17, 19, 22, 23, 26, 28

In [ ]:
subject = "pd6"

In [ ]:
def fetch_raw(subject, session):

    openneuro.download(dataset=ONDataset.dataset, target_dir=ONDataset.bids_root, include=[f"sub-{subject}"])
    bids_path = mne_bids.BIDSPath(root=ONDataset.bids_root,
                         task=ONDataset.task, suffix=ONDataset.suffix,
                         datatype=ONDataset.datatype, session=session,
                         subject=subject )

    raw = mne.io.read_raw_bdf(bids_path, verbose=verbose, preload=True)

    return raw

raw_on = fetch_raw(subject, 'on')

In [ ]:
print(raw_on.info)
print(raw_on.info["subject_info"])

In [ ]:
raw_on.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4','EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status'])

In [ ]:
raw_on.set_montage("biosemi32", verbose=verbose)

In [ ]:
raw_on.set_eeg_reference(ref_channels='average',verbose=verbose)

In [ ]:
raw_on.filter(0.5, None, fir_design='firwin', phase='zero-double', verbose=verbose)

In [ ]:
epochs_all_on = mne.make_fixed_length_epochs(raw_on, duration=3, preload=True, verbose=verbose)

In [ ]:
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, verbose=verbose)
ar.fit(epochs_all_on)
epochs_on = ar.transform(epochs_all_on)

In [ ]:
pacs_on = np.repeat(PACSettings.create_Comodulogram(raw_on.info['sfreq']), 3)

In [ ]:
pacs_on[0].fit(epochs_on.get_data(picks=['C3'])[:,0,:])

In [ ]:
pacs_on[1].fit(epochs_on.get_data(picks=['C4'])[:,0,:])

In [ ]:
pacs_on[2].comod_ = np.mean([pacs_on[0].comod_, pacs_on[1].comod_],axis=0)

In [ ]:
raw_off = fetch_raw(subject, 'off')

raw_off.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4','EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status'])
raw_off.set_montage("biosemi32", verbose=verbose)
raw_off.set_eeg_reference(ref_channels='average',verbose=verbose)
raw_off.filter(0.5, None, fir_design='firwin', phase='zero-double', verbose=verbose)

epochs_all_off = mne.make_fixed_length_epochs(raw_off, duration=3, preload=True, verbose=verbose)
ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, verbose=verbose)
ar.fit(epochs_all_off)
epochs_off = ar.transform(epochs_all_off)

calc vmax

In [ ]:
pacs_off = np.repeat(PACSettings.create_Comodulogram(raw_off.info['sfreq']), 3)                                                                                                                                                          

pacs_off[0].fit(epochs_off.get_data(picks=['C3'])[:,0,:])
pacs_off[1].fit(epochs_off.get_data(picks=['C4'])[:,0,:])
pacs_off[2].comod_ = np.mean([pacs_off[0].comod_, pacs_off[1].comod_],axis=0)

In [ ]:
np.concatenate(pacs_on, pacs_off)

In [ ]:
vmax = max([pac.comod_.max() for pac in pacs_on + pacs_off])

modulation index

In [ ]:
mis_on  = [ np.mean(pac.comod_) for pac in pacs_on ]
mis_off = [ np.mean(pac.comod_) for pac in pacs_off ]

In [ ]:
print("%s " % (subject))
print(" ON  C3:%.3E C4:%.3E AVG:%.3E" % (mis_on[0], mis_on[1], mis_on[2]))
print(" OFF C3:%.3E C4:%.3E AVG:%.3E" % (mis_off[0], mis_off[1], mis_off[2]))

matplotlib object with title

In [ ]:
fig,axs=plt.subplots(2,3,figsize=(16,8))

plt.suptitle("Subject: %s  PAC Method: %s"
             % (subject, ComodulogramSettings.method ))

plot the previously created pacs

In [ ]:
pacs_on[0].plot(titles=["ON C3 MI=%.2E"  % mis_on[0]], axs=[axs[0,0]], vmin=0, vmax=vmax)
pacs_on[1].plot(titles=["ON C4 MI=%.2E"  % mis_on[1]], axs=[axs[0,1]], vmin=0, vmax=vmax)
pacs_on[2].plot(titles=["ON AVG MI=%.2E" % mis_on[2]], axs=[axs[0,2]], vmin=0, vmax=vmax)

pacs_off[0].plot(titles=["OFF C3 MI=%.2E"  % mis_off[0]], axs=[axs[1,0]], vmin=0, vmax=vmax)
pacs_off[1].plot(titles=["OFF C4 MI=%.2E"  % mis_off[1]], axs=[axs[1,1]], vmin=0, vmax=vmax)
pacs_off[2].plot(titles=["OFF AVG MI=%.2E" % mis_off[2]], axs=[axs[1,2]], vmin=0, vmax=vmax)


In [ ]:
plt.show()